In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn import preprocessing
np.random.seed(42)

In [2]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
st_info = pd.read_csv("studs_info.csv")

In [3]:
X_all = X_train_in.append(X_val, ignore_index=True)

In [4]:
X_train_in.head(1)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг


In [5]:
student_level = []
big_group = []
small_group = []

for i in X_train_in['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_train_in['STD_LVL'] = student_level
X_train_in['big_group'] = big_group
X_train_in['small_group'] = small_group

In [6]:
# {'Дифференцированный зачет',5
#  'Зачет',1
#  'Курсовая работа',3
#  'Курсовой проект',4
#  'Экзамен' 2}
X_train_in['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [7]:
st_info = st_info.drop_duplicates(subset=['STD_ID'], keep="last")
st_info["Образование"].fillna("СР", inplace = True)
mage = []
bablo = []
for i in st_info["Дата выдачи"]:
    age = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S', errors='ignore')
    if isinstance(age, pd._libs.tslibs.timestamps.Timestamp):
        mage.append(age.today().year - age.year + 18)
    else:
        mage.append(18)
for i in st_info["Категория обучения"]:
    bablo.append(i == "БП")
st_info["age"] = mage
st_info["bablo"] = bablo
le = preprocessing.LabelEncoder()
le.fit(st_info.Образование)

st_info["learn"] = le.transform(st_info["Образование"])
le = preprocessing.LabelEncoder()
le.fit(X_all.ГОД)
X_train_in.loc[:,"exam_year"] = le.transform(X_train_in.loc[:,"ГОД"])
X_val.loc[:,"exam_year"] = le.transform(X_val.loc[:,"ГОД"])

In [8]:
le = preprocessing.LabelEncoder()

le.fit(X_all.ДИСЦИПЛИНА)

X_train_in.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_train_in.loc[:,"ДИСЦИПЛИНА"])
X_val.loc[:,"ДИСЦИПЛИНА"] = le.transform(X_val.loc[:,"ДИСЦИПЛИНА"])

In [9]:
X_train_in["mark"] = y_train_in["mark"]

In [10]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,exam_year,mark
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,5,5
72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,2,946,05,31,01,2,5
96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,2,1326,05,31,03,6,5
273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,2,1100,03,38,02,4,4
371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,2,713,05,31,01,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,1.0,2.0,3,364,03,40,01,4,3
260998,182794,31.05.01 Лечебное дело,2018-2019,2.0,4.0,2,556,05,31,01,5,3
57245,162391,31.05.01 Лечебное дело,2016-2017,1.0,2.0,5,228,05,31,01,3,4
218529,202479,38.03.01 Экономика,2017-2018,1.0,1.0,5,238,03,38,01,4,4


In [11]:
agg_features = X_train_in.groupby('STD_ID')['mark'].agg(['mean','max','min','std','sum','count', np.median, np.ptp, np.average]).reset_index()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ec2-user/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
hren = X_train_in.groupby(["НАПРАВЛЕНИЕ", "КУРС", "СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "exam_year"]).mean()["mark"]
X_train_in = X_train_in.merge(hren, on=["НАПРАВЛЕНИЕ", "КУРС", "СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "exam_year"], how="left")
difference = []
for i in zip(X_train_in["mark_x"], X_train_in["mark_y"]):
    difference.append(i[0] - i[1])
X_train_in["difference"] = difference
small_feature = X_train_in.groupby('STD_ID')['difference'].agg(['mean']).reset_index()
X_train_in = pd.merge(X_train_in,small_feature,on='STD_ID', how="left")
X_val = pd.merge(X_val,small_feature,on='STD_ID', how="left")

In [13]:
# new_info = st_info.groupby(st_info.index).first()
# new_info = new_info.fillna(0)
# new_info.head(1)

In [14]:
X_val = X_val.merge(st_info[["STD_ID", "age", "bablo", "learn"]], on="STD_ID", how="left")
X_train_in = X_train_in.merge(st_info[["STD_ID", "age", "bablo", "learn"]], how="left",left_on='STD_ID',right_on='STD_ID', validate = 'm:1')

In [15]:
X_train_in = pd.merge(X_train_in,agg_features,on='STD_ID')

In [16]:
X_train_in.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,STD_LVL,big_group,small_group,...,learn,mean_y,max,min,std,sum,count,median,ptp,average
0,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,2,408,03,38,02,...,6,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
1,175711,38.03.02 Менеджмент,2017-2018,1.0,1.0,2,731,03,38,02,...,6,4.181818,5,3,0.58849,92,22,4.0,2,4.181818
2,175711,38.03.02 Менеджмент,2017-2018,1.0,2.0,5,254,03,38,02,...,6,4.181818,5,3,0.58849,92,22,4.0,2,4.181818


In [17]:
# stud_mean_mark = X_train_in.groupby("STD_ID").mean()["mark"]
# X_train_in.loc[:,"stud_mean"] = X_train_in.loc[:,"STD_ID"].map(stud_mean_mark)

In [18]:
X_train_in.shape

(105058, 27)

In [19]:
X_val['АТТЕСТАЦИЯ'].replace(['Дифференцированный зачет','Зачет', 'Курсовая работа', 'Курсовой проект','Экзамен'],[5,1,3,4,2],inplace=True)

In [20]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,exam_year,mean,age,bablo,learn
0,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260,5,-0.092971,26,True,3
1,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028,6,0.129593,22,True,6
2,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269,5,-0.181342,20,False,6


In [21]:
X_val = pd.merge(X_val, agg_features, on='STD_ID', how='left')

In [22]:
student_level = []
big_group = []
small_group = []

for i in X_val['НАПРАВЛЕНИЕ']:
    student_level.append(i.split('.')[1])
    big_group.append(i.split('.')[0])
    small_group.append(i.split('.')[2][:2])
    
X_val['STD_LVL'] = student_level
X_val['big_group'] = big_group
X_val['small_group'] = small_group

In [23]:
X_val.head(3)

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА,exam_year,mean_x,age,...,min,std,sum,count,median,ptp,average,STD_LVL,big_group,small_group
0,163261,35.03.04 Агрономия,2018-2019,3.0,5.0,5,260,5,-0.092971,26,...,3.0,0.538958,86.0,21.0,4.0,2.0,4.095238,03,35,04
1,162990,23.03.03 Эксплуатация транспортно-технологичес...,2019-2020,4.0,7.0,2,1028,6,0.129593,22,...,3.0,0.772049,113.0,29.0,4.0,2.0,3.896552,03,23,03
2,192595,40.03.01 Юриспруденция,2018-2019,1.0,2.0,2,269,5,-0.181342,20,...,3.0,0.943880,43.0,11.0,4.0,2.0,3.909091,03,40,01


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_train_in,X_train_in["mark_x"], shuffle=True, test_size=0.01)

In [26]:
y_test.shape, y_train.shape

((31518,), (73540,))

In [27]:
from catboost import CatBoostRegressor, CatBoostClassifier

In [28]:
train_col = ["КУРС","СЕМЕСТР", "АТТЕСТАЦИЯ", "ДИСЦИПЛИНА", "mean_x", "std",
             "count", "max", "min", "STD_LVL", 'median', "sum",
             'ptp', 'average', 'big_group', 'small_group', "age", "bablo", "exam_year", "learn", "mean_y"]
# X_train["STD_ID"]= X_train["STD_ID"].astype(int)
# X_test["STD_ID"]= X_test["STD_ID"].astype(int)


In [29]:
model = CatBoostClassifier(iterations=1000,  classes_count=6, depth = 7, learning_rate=0.3,)
# model = CatBoostRegressor(iterations=1000, learning_rate=0.05,)

In [30]:
model.fit(X_train[train_col],y_train)

Found only 3 unique classes in the data, but have defined 6 classes. Probably something is wrong with data.


0:	learn: 0.9631896	total: 63.3ms	remaining: 1m 3s
1:	learn: 0.8889038	total: 78.3ms	remaining: 39.1s
2:	learn: 0.8423517	total: 92.1ms	remaining: 30.6s
3:	learn: 0.8110026	total: 106ms	remaining: 26.5s
4:	learn: 0.7867512	total: 120ms	remaining: 23.9s
5:	learn: 0.7717125	total: 134ms	remaining: 22.2s
6:	learn: 0.7615494	total: 148ms	remaining: 20.9s
7:	learn: 0.7546379	total: 161ms	remaining: 19.9s
8:	learn: 0.7480718	total: 174ms	remaining: 19.2s
9:	learn: 0.7399201	total: 188ms	remaining: 18.6s
10:	learn: 0.7350806	total: 202ms	remaining: 18.2s
11:	learn: 0.7319976	total: 215ms	remaining: 17.7s
12:	learn: 0.7285078	total: 228ms	remaining: 17.3s
13:	learn: 0.7267469	total: 241ms	remaining: 17s
14:	learn: 0.7243649	total: 254ms	remaining: 16.7s
15:	learn: 0.7218756	total: 268ms	remaining: 16.5s
16:	learn: 0.7194275	total: 281ms	remaining: 16.2s
17:	learn: 0.7177153	total: 294ms	remaining: 16s
18:	learn: 0.7160578	total: 307ms	remaining: 15.8s
19:	learn: 0.7130459	total: 320ms	remainin

164:	learn: 0.6231720	total: 2.31s	remaining: 11.7s
165:	learn: 0.6227228	total: 2.34s	remaining: 11.7s
166:	learn: 0.6224356	total: 2.36s	remaining: 11.8s
167:	learn: 0.6219978	total: 2.38s	remaining: 11.8s
168:	learn: 0.6216558	total: 2.4s	remaining: 11.8s
169:	learn: 0.6213381	total: 2.41s	remaining: 11.8s
170:	learn: 0.6209989	total: 2.42s	remaining: 11.8s
171:	learn: 0.6207146	total: 2.44s	remaining: 11.7s
172:	learn: 0.6202286	total: 2.45s	remaining: 11.7s
173:	learn: 0.6197462	total: 2.47s	remaining: 11.7s
174:	learn: 0.6194775	total: 2.48s	remaining: 11.7s
175:	learn: 0.6191746	total: 2.5s	remaining: 11.7s
176:	learn: 0.6186707	total: 2.51s	remaining: 11.7s
177:	learn: 0.6182536	total: 2.53s	remaining: 11.7s
178:	learn: 0.6178953	total: 2.54s	remaining: 11.7s
179:	learn: 0.6176525	total: 2.56s	remaining: 11.7s
180:	learn: 0.6172001	total: 2.57s	remaining: 11.6s
181:	learn: 0.6169030	total: 2.59s	remaining: 11.6s
182:	learn: 0.6165732	total: 2.6s	remaining: 11.6s
183:	learn: 0.6

328:	learn: 0.5746139	total: 4.79s	remaining: 9.76s
329:	learn: 0.5745045	total: 4.81s	remaining: 9.76s
330:	learn: 0.5743372	total: 4.83s	remaining: 9.76s
331:	learn: 0.5741914	total: 4.85s	remaining: 9.75s
332:	learn: 0.5738892	total: 4.86s	remaining: 9.74s
333:	learn: 0.5736324	total: 4.88s	remaining: 9.72s
334:	learn: 0.5734432	total: 4.89s	remaining: 9.71s
335:	learn: 0.5731201	total: 4.9s	remaining: 9.69s
336:	learn: 0.5727921	total: 4.92s	remaining: 9.68s
337:	learn: 0.5725642	total: 4.93s	remaining: 9.66s
338:	learn: 0.5722725	total: 4.95s	remaining: 9.65s
339:	learn: 0.5721079	total: 4.96s	remaining: 9.63s
340:	learn: 0.5718590	total: 4.98s	remaining: 9.62s
341:	learn: 0.5714930	total: 4.99s	remaining: 9.61s
342:	learn: 0.5712485	total: 5.01s	remaining: 9.6s
343:	learn: 0.5710667	total: 5.02s	remaining: 9.58s
344:	learn: 0.5708777	total: 5.04s	remaining: 9.56s
345:	learn: 0.5707658	total: 5.05s	remaining: 9.55s
346:	learn: 0.5706036	total: 5.07s	remaining: 9.53s
347:	learn: 0.

489:	learn: 0.5388329	total: 7.25s	remaining: 7.54s
490:	learn: 0.5386966	total: 7.26s	remaining: 7.53s
491:	learn: 0.5384195	total: 7.27s	remaining: 7.51s
492:	learn: 0.5381655	total: 7.29s	remaining: 7.49s
493:	learn: 0.5380891	total: 7.3s	remaining: 7.47s
494:	learn: 0.5379139	total: 7.31s	remaining: 7.46s
495:	learn: 0.5376035	total: 7.32s	remaining: 7.44s
496:	learn: 0.5374922	total: 7.34s	remaining: 7.42s
497:	learn: 0.5372253	total: 7.35s	remaining: 7.41s
498:	learn: 0.5371071	total: 7.36s	remaining: 7.39s
499:	learn: 0.5369411	total: 7.38s	remaining: 7.38s
500:	learn: 0.5367087	total: 7.39s	remaining: 7.36s
501:	learn: 0.5364602	total: 7.4s	remaining: 7.34s
502:	learn: 0.5363584	total: 7.42s	remaining: 7.33s
503:	learn: 0.5361660	total: 7.43s	remaining: 7.31s
504:	learn: 0.5360079	total: 7.44s	remaining: 7.29s
505:	learn: 0.5357987	total: 7.47s	remaining: 7.29s
506:	learn: 0.5355826	total: 7.5s	remaining: 7.29s
507:	learn: 0.5353315	total: 7.51s	remaining: 7.28s
508:	learn: 0.5

653:	learn: 0.5070195	total: 9.52s	remaining: 5.04s
654:	learn: 0.5069083	total: 9.54s	remaining: 5.03s
655:	learn: 0.5067934	total: 9.57s	remaining: 5.02s
656:	learn: 0.5067050	total: 9.59s	remaining: 5s
657:	learn: 0.5064934	total: 9.6s	remaining: 4.99s
658:	learn: 0.5062838	total: 9.62s	remaining: 4.97s
659:	learn: 0.5060282	total: 9.63s	remaining: 4.96s
660:	learn: 0.5058152	total: 9.64s	remaining: 4.95s
661:	learn: 0.5055099	total: 9.66s	remaining: 4.93s
662:	learn: 0.5053653	total: 9.67s	remaining: 4.92s
663:	learn: 0.5051914	total: 9.69s	remaining: 4.9s
664:	learn: 0.5050876	total: 9.7s	remaining: 4.89s
665:	learn: 0.5048719	total: 9.72s	remaining: 4.87s
666:	learn: 0.5047549	total: 9.73s	remaining: 4.86s
667:	learn: 0.5045588	total: 9.74s	remaining: 4.84s
668:	learn: 0.5044944	total: 9.76s	remaining: 4.83s
669:	learn: 0.5044519	total: 9.78s	remaining: 4.82s
670:	learn: 0.5043027	total: 9.8s	remaining: 4.8s
671:	learn: 0.5042186	total: 9.81s	remaining: 4.79s
672:	learn: 0.503947

818:	learn: 0.4794303	total: 11.9s	remaining: 2.64s
819:	learn: 0.4792231	total: 12s	remaining: 2.62s
820:	learn: 0.4790949	total: 12s	remaining: 2.61s
821:	learn: 0.4789040	total: 12s	remaining: 2.59s
822:	learn: 0.4787803	total: 12s	remaining: 2.58s
823:	learn: 0.4786082	total: 12s	remaining: 2.56s
824:	learn: 0.4784883	total: 12s	remaining: 2.55s
825:	learn: 0.4783823	total: 12s	remaining: 2.54s
826:	learn: 0.4782803	total: 12.1s	remaining: 2.52s
827:	learn: 0.4780731	total: 12.1s	remaining: 2.51s
828:	learn: 0.4778317	total: 12.1s	remaining: 2.49s
829:	learn: 0.4775522	total: 12.1s	remaining: 2.48s
830:	learn: 0.4773913	total: 12.1s	remaining: 2.46s
831:	learn: 0.4772139	total: 12.1s	remaining: 2.45s
832:	learn: 0.4771120	total: 12.1s	remaining: 2.43s
833:	learn: 0.4770214	total: 12.2s	remaining: 2.42s
834:	learn: 0.4767528	total: 12.2s	remaining: 2.4s
835:	learn: 0.4765704	total: 12.2s	remaining: 2.39s
836:	learn: 0.4764573	total: 12.2s	remaining: 2.38s
837:	learn: 0.4763254	total

989:	learn: 0.4551388	total: 14.6s	remaining: 148ms
990:	learn: 0.4550620	total: 14.6s	remaining: 133ms
991:	learn: 0.4548872	total: 14.6s	remaining: 118ms
992:	learn: 0.4547466	total: 14.7s	remaining: 103ms
993:	learn: 0.4546548	total: 14.7s	remaining: 88.6ms
994:	learn: 0.4545929	total: 14.7s	remaining: 73.8ms
995:	learn: 0.4543936	total: 14.7s	remaining: 59.1ms
996:	learn: 0.4542157	total: 14.7s	remaining: 44.3ms
997:	learn: 0.4540356	total: 14.7s	remaining: 29.5ms
998:	learn: 0.4538716	total: 14.8s	remaining: 14.8ms
999:	learn: 0.4537298	total: 14.8s	remaining: 0us


In [31]:
mean_absolute_error(y_test, model.predict(X_test[train_col])), r2_score(y_test, model.predict(X_test[train_col]))

(0.3407259343866997, 0.328806296980677)

In [445]:
y_pred = pd.read_csv("sample_submission.csv",index_col=0)
y_pred["mark"] = model.predict(X_val[train_col])

In [446]:
y_pred[y_pred["mark"]<3]

,mark
index,


In [447]:
#y_pred[y_pred["mark"]<3] = 3

In [448]:
#print(model.predict(X_val[train_col]).shape, pd.read_csv("sample_submission.csv",index_col=0).shape)

In [449]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'Submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('Submissions'):
    os.makedirs('Submissions')

print(submission_path)
y_pred.to_csv(submission_path, index=True)

Submissions/1585327750.csv
